In [1]:
import numpy as np
import cv2

from sklearn.metrics import pairwise  #for distance calculation

In [2]:
background = None

accumulated_weight = 0.5  #the weight is calculated before we place the hand in, and hence we can detect when it comes in by any change in the moving average of the actual frame which is precalculated

#set up the ROI for the hand
roi_top = 20
roi_bottom = 300
roi_left = 600
roi_right = 300

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    
    global background # Given a frame and a previous accumulated weight, computed the weighted average of the image passed in.
    
    if background is None:  #first time only
        background = frame.copy().astype("float")
        return None
    
    #update the bg with the running average
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
#segment the hand region in the frame

def segment(frame, threshold_min = 25):
    
    #abs diff between the bg and the image passed in - focus on the new object that has entered the prev bg with accum weights
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY) 
    
    # Apply a threshold to the image so we can grab the foreground
    image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Grab the external contours form the image
    
    if len(contours) == 0:
        return None
    
    else: #the largest ext contour should be the hand here and that is our segment
        
        hand_segment = max(contours, key = cv2.contourArea)
        
        return (thresholded, hand_segment)

In [5]:
def count_fingers(thresholded, hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment) # Calculated the convex hull of the hand segment
    
    # Now the convex hull will have at least 4 most outward points, on the top, bottom, left, and right.
    # Let's grab those points by using argmin and argmax. Keep in mind, this would require reading the documentation
    # And understanding the general array shape returned by the conv hull.
    
    top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2  #center X and center Y coord

    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull

    # Calculate the Euclidean Distance between the center of the hand and the left, right, top, and bottom.
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    max_distance = distance.max()
    
    #draw a circle with 90% radius of the max euclidean dist
    radius = int(0.9 * max_distance)
    circumference = (2 * np.pi * radius)
    
    #draw a circular ROI o
    circular_roi = np.zeros(thresholded.shape[:2], dtype = "uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    # Using bit-wise AND with the cirle ROI as a mask.
    # This then returns the cut out obtained using the mask on the thresholded hand image.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    #get all the contours in the circular ROI
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    #num of fingers
    count = 0
    
    for cnt in contours:
        
        (x, y, w, h) = cv2.boundingRect(cnt)
        
        #check conditions for a finger
        
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        #y + h is the box dims for the finger, center should be well away from it and BELOW, if it is above, ignore
        
        # 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points = ((circumference * 0.25) > cnt.shape[0])   
        
        if out_of_wrist and limit_points:
            
            count += 1
            
            
    return count    
        
    
    

In [14]:
cap = cv2.VideoCapture(0)

#we need to calc the avg bg accum weights by running avg and update the bg

num_frames = 0

while True:
    
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1) #mirror avoidance
    
    frame_copy = frame.copy()
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "Wait please, bg loading", (175, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            #cv2.imshow("Finger Count", frame_copy)
            
    else:
        
         # now that we have the background, we can segment the hand.
            
        #segment the hand region
        
        hand = segment(gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0), 1)
            fingers = count_fingers(thresholded, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 2)
            cv2.imshow("Thresholded", thresholded)
            
    
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    num_frames += 1
    cv2.imshow("Finger count", frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
        
cap.release()
cv2.destroyAllWindows()     